In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Data readed from the Wikipedia article...
import requests
import urllib.request
from lxml import html
from bs4 import BeautifulSoup
import re

# URL for wikipedia article with a list of all neighborhoods by district
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data_text = requests.get(wiki_url).content #text
#print(data_text)

# Web Page to string
raw_text = html.fromstring(data_text)

In [3]:
# To Extract Information

# 1st: get all poosible names, under the 'td' cathegory in the HTML
td_list = []
for s in raw_text.iter():
    if s.tag == 'td':
        # Save in the list all text (in special format)
        #  inside the td category
        td_list.append(s)
        #print(s.text_content())

# 2nd : Filter names
neighborhoods = []
i = 0
s = [0, 0, 0]
for td_e in td_list:
    # Convert each element in the list to a string
    text = str(td_e.text_content())
    # Save in a list all columns in the table (3 at the time)
    #  and ignorig the last two characters, all escape elements
    #  usually '\n'
    s[i] = text[0:-1]
    #print(i, s[i])
    i += 1
    # Make the inspection each 3 strings: We have a table with 3 Cols.
    if i == 3:
        # Save the neighborhood if the Borough is assigned
        # (and there is one) and the neighborhood is long enough
        # (some codes are readed, that not correspond to the table)
        if s[1] != 'Not assigned' and s[1] != '' and len(s[2]) > 3:
            #print(s)
            neighborhoods.append(s)
        if s[1] != 'Not assigned' and s[2] == 'Not assigned':
            s[2] = s[1]
            neighborhoods.append(s)
        s = [0, 0, 0]
        i = 0

# 3rd: Transform to a data frame, and rename the columns
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_Toronto = pd.DataFrame(neighborhoods, columns=cols)
df_Toronto.sort_values('PostalCode').reset_index()
df_Toronto.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df_Toronto.shape

(103, 3)

In [5]:
API_KEY_GOOGLE = ''
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

In [9]:
!pip install geocoder


     |████████████████████████████████| 102kB 7.7MB/s ta 0:00:011


In [34]:
import geocoder

# Function to get the geographical coordinates for different postal 
#  codes in Toronto
def getCoords(postCode):
    ll_coords = None
    # Loop until get the coordinates
    i = 0
    while(ll_coords == None):
        i += 1
        address = '{}, Toronto'.format(postCode)
        g = geocoder.google(address, key=API_KEY_GOOGLE)
        ll_coords = g.latlng
    return ll_coords

In [ ]:
# Loop for each post code, to get its coordinates

latAll = []
lonAll = []
for n in range(df_Toronto.shape[0]):
    postCode = df_Toronto['PostalCode'][n]
    ll_coords = getCoords(postCode)
    lat = ll_coords[0]
    lon = ll_coords[1]
    latAll.append(lat)
    lonAll.append(lon)
    print('Postal Code: {}, Coordinates: {}'.format(postCode, ll_coords))
    
df_Toronto['Latitude'] = latAll
df_Toronto['Longitude'] = lonAll
df_Toronto.head(15)

In [29]:
df_Toronto.shape

(103, 3)

In [30]:
CLIENT_ID = 'rajesi32@in.ibm.com' # your Foursquare ID
CLIENT_SECRET = 'tunnababu' # your Foursquare Secret
VERSION = '20182106' # Foursquare API version
LIMIT = 100

In [31]:
import json
from pandas.io.json import json_normalize

# Function to get the nearby venues for a set of locations

def getNearbyVenues(names, latAll, lonAll, radius=500):
    venue_list = []
    for name, lat, lon in zip(names, latAll, lonAll):
        #print(name)
        # Create the API and request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            LIMIT)
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venue_list.append([(
            name, 
            lat, 
            lon, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
        nearby_venues.columns = ['PostalCode', 
                  'PC Latitude', 
                  'PC Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
    # Return
    return(nearby_venues)


In [32]:
M_venues = getNearbyVenues(names=df_Toronto['PostalCode'],
                           latAll=df_Toronto['Latitude'],
                           lonAll=df_Toronto['Longitude'])

print('Nearby venues done')

M_venues.head(20)

KeyError: 'Latitude'

In [19]:

#First, group by Postal Code
M_venues.groupby('PostalCode').count()

# One-hot enconding, and adding the Postal Code back to the Data Frame
M_oneHot = pd.get_dummies(M_venues[['Venue Category']], prefix="", prefix_sep="")
M_oneHot['PostalCode'] = M_venues['PostalCode']

# Move Postal Code column to the first column
fixed_columns = [M_oneHot.columns[-1]] + list(M_oneHot.columns[:-1])
M_oneHot = M_oneHot[fixed_columns]

M_oneHot.head(10)

NameError: name 'M_venues' is not defined

In [20]:
M_group = M_oneHot.groupby('PostalCode').mean().reset_index()
M_group

NameError: name 'M_oneHot' is not defined

In [21]:
# See all top 5 of venues for each Postal Code

numMax = 5
for pc in M_group['PostalCode']:
    print('-----'+pc+'-----')
    temp = M_group[M_group['PostalCode'] == pc].T.reset_index()
    temp.columns = ['Venue', 'Freq.']
    temp = temp.iloc[1:]
    temp['Freq.'] = temp['Freq.'].astype(float)
    temp = temp.round({'Freq.':2})
    print(temp.sort_values('Freq.', ascending=False).reset_index(drop=True).head(numMax))
    print('\n')

    

NameError: name 'M_group' is not defined

In [22]:
from sklearn.cluster import KMeans

# Number of Clusters. Lets start with 5, as in the NYC example
Kmax = 7

M_group_clustering = M_group.drop('PostalCode', 1)

# k-Means Clustering
kmM = KMeans(n_clusters=Kmax, random_state=0).fit(M_group_clustering)

# Check the cluster labels
kmM.labels_[0:10]

NameError: name 'M_group' is not defined

In [23]:
numTop = 10

indicators = ['st', 'nd', 'rd']

# Function to sort venues in descending order
def mostCommon(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top]

# Create columns according to number of top venues
columns = ['PostalCode']
for indx in np.arange(numTop):
    try:
        columns.append('{}{} Most Common Venue'.format(indx+1, indicators[indx]))
    except:
        columns.append('{}th Most Common Venue'.format(indx+1))

# Create a new dataframe
venuesSort = pd.DataFrame(columns=columns)
venuesSort['PostalCode'] = M_group['PostalCode']

for indx in np.arange(M_group.shape[0]):
    venuesSort.iloc[indx, 1:] = mostCommon(M_group.iloc[indx, :], numTop)
venuesSort.head()

NameError: name 'M_group' is not defined

In [24]:
venuesSort.insert(0, 'Cluster Labels', kmM.labels_)

# Merge the original data frame with the cluster data and venues
M_merged = df_Toronto
M_merged = M_merged.join(venuesSort.set_index('PostalCode'), on='PostalCode')

# There are some NaNs in the clustering, maybe because there are
#  no data from the venues in such neighborhood!
M_merged.dropna(axis=0, inplace=True)

# For some reason, after the merging the Cluster labels are float...
M_merged['Cluster Labels'].astype('int32')

M_merged.head(10)

NameError: name 'kmM' is not defined

In [25]:
M_merged.dtypes

NameError: name 'M_merged' is not defined

In [26]:
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Mlat = 43.741667
Mlon = -79.373333

# Constructor for the map
mapClus = folium.Map(location=[Mlat, Mlon],
                     zoom_start=10)

# Map colors
x = np.arange(numMax)
ys = [i + x + (i*x)**2 for i in range(Kmax)]
colorsArray = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colorsArray]

# Add markers to the map
markers_col = []
for lat, lon, pc, clus in zip(M_merged['Latitude'],
                              M_merged['Longitude'],
                              M_merged['PostalCode'],
                              M_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ' Cluster ' + str(clus), parse_html=True)
    j = int(clus)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[j-1],
        fill=True,
        fill_color=rainbow[j-1],
        fill_opacity=0.7    
    ).add_to(mapClus)

mapClus

ModuleNotFoundError: No module named 'folium'